# Retrogate Analysis

In [ ]:
%run ./nmm-game.ipynb
%run ./nmm-symmetry.ipynb

import pymongo
import math

localMongo = pymongo.MongoClient("mongodb://localhost:27017/")
retroDb = localMongo["retro-database"]
retroCol = retroDb["retro-collection"]

## Generierung der Spielfelder

Die Endspieldatenbank soll alle Zustände speichern, in denen der weiße Spieler auf jeden Fall gewinnt. Dafür ist es notwendig, zunächst alle möglichen Spielfelder mit drei weißen und drei schwarzen Spielsteinen zu generieren, um im Nachhinein untersuchen zu können, ob diese zu den Gewinnzuständen gehören.

Die Funktion `generateBoards` nimmt ein leeres Spielfeld und befüllt dieses Stein für Stein. Dabei wird jeder der sechs Steine auf jede der 24 Positionen des Spielfeldes gesetzt. So wird sichergestellt, dass auch wirklich jede mögliche Kombination erstellt wird.
Da es nicht ausgeschlossen werden kann, dass zwei Steine auf die selbe Position gesetzt werden und somit nicht die gewünschte 3-3-Kombination vorhanden ist, wird vor dem Speichern der Spielbretter überprüft, ob für beide Farben drei Steine platziert wurden.

Da es für das Mühlespiel egal ist, ob beispielsweise der 1. oder der 3. weiße Stein auf der Position `(1, 4)` liegt, werden die Spielbretter in ein globales `set` gespeichert. Dadurch wird sichergestellt, dass ein Spielbrett nicht zwei mal abgespeichert ist.

Die Entscheidung, die Spielbretter global abzuspeichern wurde bewusst getroffen, um bei Änderungen an anderen Funktionen nicht die Generierung der Spielbretter neu starten zu müssen, welche einiges an Zeit benötigt.

In [ ]:
startBoard = ((' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '),
    (' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '),
    (' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '))
boards = set()

def generateBoards(board=startBoard):
    global boards
    count = 0
    for whiteOneX in range(0,3):
        for whiteOneY in range(0, 8):
            count += 1
            print('Progress: ' + str(count) + '/24')
            for blackOneX in range(0,3):
                for blackOneY in range(0, 8):
                    for whiteTwoX in range(0,3):
                        for whiteTwoY in range (0, 8):
                            for blackTwoX in range(0,3):
                                for blackTwoY in range(0, 8):
                                    for whiteThreeX in range(0,3):
                                        for whiteThreeY in range(0,8):
                                            for blackThreeX in range(0,3):
                                                for blackThreeY in range(0,8):
                                                    board1 = place(board, (whiteOneX, whiteOneY), 'w')
                                                    board2 = place(board1, (blackOneX, blackOneY), 'b')
                                                    board3 = place(board2, (whiteTwoX, whiteTwoY), 'w')
                                                    board4 = place(board3, (blackTwoX, blackTwoY), 'b')
                                                    board5 = place(board4, (whiteThreeX, whiteThreeY), 'w')
                                                    board6 = place(board5, (blackThreeX, blackThreeY), 'b')
                                                    white = countStones(('', board6), 'w')
                                                    black = countStones(('', board6), 'b')
                                                    if (white == 3 and black == 3):
                                                        boards.add(board6)

In [ ]:
def getWeightForState(state, minWeight = math.inf):
    weight = 0
    counter = 1
    for r in range(3):
        for c in range(8):
            weight += (2**(counter if state[1][r][c] == 'w' else counter + 24) if state[1][r][c] != ' ' else 0)
            counter += 1
            if (weight > minWeight):
                return weight
    return weight

In [ ]:
def getUniqueStateForSymmetry(state):
    symmetries = findSymmetries(state)
    
    state = None
    minWeight = math.inf 
    for s in symmetries:
        weight = getWeightForState(s, minWeight)
        if weight < minWeight:
            state = s
            minWeight = weight
    return state

Die Funktion `getSymmetryUnique(boards)` iteriert über jedes Spielfeld und erhält von der `getUniqueStateForSymmetry(state)` das Spielfeld, welches unter allen Symmetrien die niedrigste Gewichtung (`weight`) aufweist und fügt dieses den `uniqueStates` hinzu. So wird sichergestellt, das für alle Symmetrien nur ein Repräsentant abgespeichert wird.

In [ ]:
def getSymmetryUnique(boards):
    uniqueStates = set()
    stones = (0, 0)
    for board in boards:
        state = tuple([stones, board])
        uniqueStates.add(getUniqueStateForSymmetry(state))
    return uniqueStates

In [ ]:
def fillDb(states):
    fullStep = []
    stepCount = 0
    
    stepCount += 1
    
    fullStates = states.copy()
    halfStates = states.copy()
    _states = fullStates.copy()

    for state in _states:
        _, board = state
        if (findPossibleMills(board, 'w')) == set():
            continue;
        _nextStates = nextStates(state, 'w')
        for ns in _nextStates:
            if (finished(ns, 'w') and utility(ns, 'w') == 1):
                entry = { "state": state, "nextState": ns, "steps": stepCount }
                retroCol.insert_one(entry)
                fullStep.append(state)
                fullStates.remove(state)
                break
    
    while len(states) > 0:
        enrichedFullStep = set()
        for s in fullStep:
            enrichedFullStep |= findSymmetries(s)
        
        _states = halfStates.copy()
        halfStep = []
        stepCount += 1
        print('current step-depth: ' + str(stepCount))
        
        for state in _states:
            _nextStates = nextStates(state, 'b')
            if (_nextStates.issubset(enrichedFullStep)):
                halfStep.append(state)
                halfStates.remove(state)
                    
        if len(halfStep) == 0:
            break
        
        enrichedHalfStep = set()
        for s in halfStep:
            enrichedHalfStep |= findSymmetries(s)
        
        _states = fullStates.copy()
        
        stepCount += 1
        print('current step-depth: ' + str(stepCount))
        
        for state in _states:
            _nextStates = nextStates(state, 'w')
            for ns in _nextStates:
                if (ns in enrichedHalfStep):
                    entry = { "state": state, "nextState": ns, "steps": stepCount }
                    retroCol.insert_one(entry)
                    fullStep.append(state)
                    fullStates.remove(state)
                    break

In [ ]:
generateBoards()
global boards
states = getSymmetryUnique(boards)
fillDb(states)